In [11]:
# -----------------------------
#  Movie Recommendation System
# Author: Ratikanta Pati
# -----------------------------

# --- 1. Install & Import Dependencies ---
!pip install scikit-learn tqdm --quiet

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from tqdm import tqdm
import zipfile, os, urllib.request

# --- 2. Download MovieLens Dataset Automatically ---
DATA_URL = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
DATA_ZIP = "ml-latest-small.zip"

if not os.path.exists("ml-latest-small"):
    print(" Downloading MovieLens dataset...")
    urllib.request.urlretrieve(DATA_URL, DATA_ZIP)
    with zipfile.ZipFile(DATA_ZIP, 'r') as zip_ref:
        zip_ref.extractall(".")
    print(" Dataset downloaded and extracted!")

MOVIES_CSV = "ml-latest-small/movies.csv"
RATINGS_CSV = "ml-latest-small/ratings.csv"

# --- 3. Load Data ---
def load_data():
    print("\n Loading data...")
    movies = pd.read_csv(MOVIES_CSV)
    ratings = pd.read_csv(RATINGS_CSV)
    print(f"Movies loaded: {len(movies)}, Ratings loaded: {len(ratings)}")
    return movies, ratings

# --- 4. TF-IDF Matrix for Content-Based Filtering ---
def build_tfidf_matrix(movies_df):
    print("\n Building TF-IDF matrix...")
    movies_df['content'] = movies_df['title'].fillna('') + ' ' + movies_df['genres'].fillna('')
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movies_df['content'])
    print(" Cosine similarity for content calculated.")
    content_sim = cosine_similarity(tfidf_matrix, dense_output=False)
    return tfidf, content_sim

# --- 5. Sparse Matrix for Collaborative Filtering ---
def build_sparse_item_user_matrix(ratings_df, movies_df):
    print("\n Building item-user matrix...")
    movie_index = {mid: i for i, mid in enumerate(movies_df['movieId'])}
    user_ids = ratings_df['userId'].unique()
    user_index = {uid: i for i, uid in enumerate(user_ids)}

    row_idx = ratings_df['movieId'].map(movie_index)
    col_idx = ratings_df['userId'].map(user_index)
    data = ratings_df['rating']

    sparse_matrix = csr_matrix((data, (row_idx, col_idx)), shape=(len(movies_df), len(user_ids)))
    print(" Calculating cosine similarity between movies...")
    item_sim = cosine_similarity(sparse_matrix, dense_output=False)
    return sparse_matrix, item_sim, movie_index

# --- 6. Content-Based Recommendation ---
def recommend_content_based(title, movies_df, content_sim):
    if title not in movies_df['title'].values:
        print(f" Movie '{title}' not found.")
        return pd.DataFrame()
    idx = movies_df[movies_df['title'] == title].index[0]
    sim_scores = list(enumerate(content_sim[idx].toarray().flatten()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:11]]
    return movies_df.iloc[top_indices][['title', 'genres']]

# --- 7. Collaborative Recommendation ---
def recommend_collaborative(user_id, movies_df, ratings_df, item_sim, movie_index):
    if user_id not in ratings_df['userId'].unique():
        print(f" User ID {user_id} not found.")
        return pd.DataFrame()
    user_ratings = ratings_df[ratings_df['userId'] == user_id]
    rated_movies = user_ratings['movieId'].tolist()
    scores = np.zeros(len(movies_df))
    for _, row in user_ratings.iterrows():
        mid, rating = row['movieId'], row['rating']
        if mid in movie_index:
            idx = movie_index[mid]
            sim_vector = item_sim[idx].toarray().flatten()
            scores += sim_vector * rating
    movies_df['score'] = scores
    recommendations = movies_df[~movies_df['movieId'].isin(rated_movies)].sort_values('score', ascending=False)
    return recommendations[['title', 'genres', 'score']].head(10)

# --- 8. Hybrid Recommendation ---
def recommend_hybrid(user_id, title, movies_df, ratings_df, content_sim, item_sim, movie_index, alpha=0.6):
    print(f"\n⚡ Generating hybrid recommendations for user {user_id} and movie '{title}'...")
    content_recs = recommend_content_based(title, movies_df, content_sim)
    collab_recs = recommend_collaborative(user_id, movies_df, ratings_df, item_sim, movie_index)
    combined = pd.merge(content_recs, collab_recs, on='title', how='outer', suffixes=('_content', '_collab'))
    combined['hybrid_score'] = combined['score'].fillna(0) * alpha
    combined = combined.sort_values('hybrid_score', ascending=False)
    return combined[['title', 'genres_collab', 'hybrid_score']].head(10)

# --- 9. Build Everything ---
def build_all():
    movies, ratings = load_data()
    tfidf, content_sim = build_tfidf_matrix(movies)
    sparse_matrix, item_sim, movie_index = build_sparse_item_user_matrix(ratings, movies)
    return {'movies': movies, 'ratings': ratings, 'content_sim': content_sim,
            'item_sim': item_sim, 'movie_index': movie_index}

# --- 10. Example Execution ---
print("\n🎬 Building Movie Recommendation System...")
model = build_all()

movies, ratings = model['movies'], model['ratings']
content_sim, item_sim, movie_index = model['content_sim'], model['item_sim'], model['movie_index']

print("\n🔹 Content-Based Recommendations for 'Toy Story (1995)'")
display(recommend_content_based("Toy Story (1995)", movies, content_sim))

print("\n🔹 Collaborative Recommendations for User 1")
display(recommend_collaborative(1, movies, ratings, item_sim, movie_index))

print("\n🔹 Hybrid Recommendations (User 1 + Toy Story (1995))")
display(recommend_hybrid(1, "Toy Story (1995)", movies, ratings, content_sim, item_sim, movie_index))



🎬 Building Movie Recommendation System...

 Loading data...
Movies loaded: 9742, Ratings loaded: 100836

 Building TF-IDF matrix...
 Cosine similarity for content calculated.

 Building item-user matrix...
 Calculating cosine similarity between movies...

🔹 Content-Based Recommendations for 'Toy Story (1995)'


,title,genres
2355,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
7355,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX
3595,"Toy, The (1982)",Comedy
2539,We're Back! A Dinosaur's Story (1993),Adventure|Animation|Children|Fantasy
26,Now and Then (1995),Children|Drama
4089,Toy Soldiers (1991),Action|Drama
1617,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy
6194,"Wild, The (2006)",Adventure|Animation|Children|Comedy|Fantasy
1,Jumanji (1995),Adventure|Children|Fantasy
12,Balto (1995),Adventure|Animation|Children



🔹 Collaborative Recommendations for User 1


,title,genres,score
2195,Ferris Bueller's Day Off (1986),Comedy,352.684010
793,Die Hard (1988),Action|Crime|Thriller,330.949884
1445,"Breakfast Club, The (1985)",Comedy|Drama,328.180507
1158,"Fifth Element, The (1997)",Action|Adventure|Comedy|Sci-Fi,326.437374
902,Aliens (1986),Action|Adventure|Horror|Sci-Fi,323.117648
1071,Mars Attacks! (1996),Action|Comedy|Sci-Fi,322.292920
2078,"Sixth Sense, The (1999)",Drama|Horror|Mystery,322.198573
2014,Austin Powers: The Spy Who Shagged Me (1999),Action|Adventure|Comedy,316.072858
706,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,313.873640
507,Terminator 2: Judgment Day (1991),Action|Sci-Fi,313.420152



🔹 Hybrid Recommendations (User 1 + Toy Story (1995))

⚡ Generating hybrid recommendations for user 1 and movie 'Toy Story (1995)'...


,title,genres_collab,hybrid_score
6,Ferris Bueller's Day Off (1986),Comedy,211.610406
5,Die Hard (1988),Action|Crime|Thriller,198.569931
4,"Breakfast Club, The (1985)",Comedy|Drama,196.908304
7,"Fifth Element, The (1997)",Action|Adventure|Comedy|Sci-Fi,195.862424
1,Aliens (1986),Action|Adventure|Horror|Sci-Fi,193.870589
9,Mars Attacks! (1996),Action|Comedy|Sci-Fi,193.375752
12,"Sixth Sense, The (1999)",Drama|Horror|Mystery,193.319144
2,Austin Powers: The Spy Who Shagged Me (1999),Action|Adventure|Comedy,189.643715
0,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,188.324184
13,Terminator 2: Judgment Day (1991),Action|Sci-Fi,188.052091


# New Section